<a href="https://colab.research.google.com/github/ipek-yilmaz/Google-Brain---Ventilator-Pressure-Prediction/blob/master/Google_Pressure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("../input/ventilator-pressure-prediction/train.csv")

In [ ]:
df.head(60)

In [ ]:
df.drop(["id","breath_id"], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
#Checking the nan data
df.isnull().sum()

In [ ]:
#drop nan values in train_set
df.dropna(axis=0, inplace=True)

In [ ]:
#Checking the data's type
df.info()
# All of them as an integer format

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().round(2)

In [ ]:
fig = plt.subplots(figsize=(6,5), dpi=90)
sns.boxplot(y=df["u_in"], width=0.15)
plt.title("The control input for the inspiratory solenoid valve percantage")

In [ ]:
sns.heatmap(df.corr(), cmap="viridis", annot=True)

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(15,12))

for indis,label in enumerate(['R', 'C', 'time_step', 'u_in', 'u_out', 'pressure']):
  plt.subplot(2,3,indis+1)
  sns.histplot(df[label])
  plt.title("{} distribution".format(label))

In [ ]:
#Show the R and C value 
plt.figure(figsize=(12,6))

for indis, col in enumerate(['R', 'C']):
  plt.subplot(1,2,indis+1)
  sns.countplot(df[col], palette="viridis")

In [ ]:
df

In [ ]:
df_train = df.iloc[:, 5:6].values

In [ ]:
df_train

In [ ]:
len(df_train)

### Applying Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
ms = MinMaxScaler()
train_set_with_s = ms.fit_transform(df_train) 

### Creating a data structure with 50 timesteps and 1 output

In [ ]:
X_train = []
y_train = []

for i in range(50,len(train_set_with_s)):
  X_train.append(train_set_with_s[i-50:i, 0])
  y_train.append(train_set_with_s[i,0])

X_train, y_train = np.array(X_train), np.array(y_train)

## Reshaping

In [ ]:
X_train

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

### Building the LSTM

In [ ]:
#importing keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM,Dense, Activation, Dropout
from keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()

### Adding first LSTM layers and some dropout

In [ ]:
model.add(LSTM(units=20, activation="selu", return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

### Adding second LSTM layers and some dropout

In [ ]:
model.add(LSTM(units=30, activation="selu",return_sequences=True ))
model.add(Dropout(0.2))

### Adding third LSTM layers and some dropout

In [ ]:
model.add(LSTM(units=40, activation="selu", return_sequences=True))
model.add(Dropout(0.3))

### Adding fourth LSTM layers and some dropout

In [ ]:
model.add(LSTM(units=50, activation="selu"))
model.add(Dropout(0.3))

## Adding the output layer

In [ ]:
model.add(Dense(units=1))

### Compiling the LSTM

In [ ]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
earlyStopping = EarlyStopping(monitor="val_loss",mode="min",verbose=1,patience=25)

In [ ]:
model.fit(x=X_train, y=y_train, epochs = 1, batch_size=64, verbose = 1, callbacks=[earlyStopping])

### Making predictions and visualizing the results

In [ ]:
df_test = pd.read_csv("../input/ventilator-pressure-prediction/test.csv")

In [ ]:
df_test

In [ ]:
df_test.dropna(axis=0, inplace=True)

In [ ]:
test_pressure_value = df_test.iloc[:, 2:].values

In [ ]:
""""
df_test = np.array(df_test)
df_test= np.array(df_test)[indices.astype(int)]
df_test.reshape((df_test, df_test[0], df_test[1], 1))
""""

In [ ]:
type(df_test[0])

In [ ]:
y_pred = model.predict(test_pressure_value)

In [ ]:
#Concat the real value and predicted value
df_total = pd.concat((df["pressure"], y_pred), axis=0)
inputs = df_total[len(df_total) - len(y_pred) - 50: ].values
inputs.reshape(-1,1)
inputs = sc.transform(inputs)

In [ ]:
#Visualizing the results
plt.plot(df["pressure"], color="red", label="Real Pressure Value")
plt.plot(y_pred, color="blue", label="Predicted Pressure Value")
plt.xlabel("Time")
plt.ylabel("Pressure")
plt.title("Pressure Value Prediction")
plt.legend()

The loss value of the model was low as I expected, but I encountered a reshape error while testing the model. Please, you can freely discuss about model and you can upvote

The model was created only taking the pressure value so it made more sense to use another model with lstm(using the power of ensemble technique). Enjoy data science journey   